In [1]:

import os
import cv2
from os.path import join,abspath
from natsort import natsorted
import numpy as np
from glob import glob
from time import time

In [2]:
from path import *


In [3]:
for key,value in paths.items():
    paths[key]=join('..',value)

In [4]:
paths

{'Data': '../Data',
 'original': '../Data/original',
 'custom': '../Data/custom',
 'neg_custom': '../Data/custom/n',
 'custom_p': '../Data/custom/p',
 'custom_p_train': '../Data/custom/p/train',
 'custom_p_test': '../Data/Test/custom',
 'sample': '../Data/custom/sample',
 'public': '../Data/Public',
 'neg_public': '../Data/Public/Negative',
 'celeba': '../Data/Public/celeba',
 'public_p': '../Data/Public/p',
 'public_p_train': '../Data/Public/p/train',
 'public_p_test': '../Data/Test/public',
 'res': '../Result',
 'res_cus': '../Result/Custom',
 'res_pub': '../Result/Public',
 'src': '../src',
 'utils': '../utils',
 'turtorial': '../turtorial'}

### Loading custom trained Haar classifer

In [5]:
haar_cascade = cv2.CascadeClassifier('../cascade.xml')

### Loading pretrained frontal face haar cascade classifier
>Manually counting the number of faces in large samples of images creates errors.**Thus to obtain the ground truth related to number_of_faces we will be using pretrained cascade classifier xml file.** 

In [6]:
pretrained_cascade=cv2.CascadeClassifier('../haarcascade_frontalface_alt.xml')

## Preprocessing the images 

In [7]:
## convert to Grayscale Image   ve_dire,img)
def bgr2gray(img):
   return cv2.cvtColor(img, cv2.COLOR_BGR2GRAY) 

## Defining funtion required for testing
> Below is the function to count the total number of faces from a given test directory. It will return total number of faces detected from the given directory as well the amount of time it takes to detect those faces.

In [15]:
### Getting the total number of faces within images folder

def test(classifier,dire):
    face_count=0
    start_time=time()
    img_detected=0
    for img_dire in dire:
        img=cv2.imread(img_dire)
        #img_name=img_dire.split('/')[-1].split('.')[0]
        gray_img=bgr2gray(img)
        faces_rect = classifier.detectMultiScale(gray_img, scaleFactor=1.1, minNeighbors=9)
        if len(faces_rect)!=0:
            img_detected+=1
        for (x, y, w, h) in faces_rect:
            #print('ok')
            face_count+=1
    elapsed_time=time()-start_time
    return face_count,elapsed_time,img_detected
    

### Calculating the Accuracy and Processing time of Custom Classifier

In [9]:
##Calculating the Accuracy of the cascade classifier. 


def get_acc(g_t,pred): ## if cust_dataset g_t =number of faces detected by pretrained model. for public dataset, g_t=len(images) in a folder
    error=np.absolute(g_t-pred)
    error_per=(error/g_t)*100
    acc_per=(100-error_per)
    return acc_per

In [10]:
## Calculating the precision time

def time_per_face(face_count,elapsed_time):
    return (elapsed_time/face_count)

def time_per_image(num_image,elapsed_time):
    return (elapsed_time/num_image)

### Function the minimum size of detected face size 

In [11]:
def minFace(img_dires):
    min_siz=[]
    for img_dire in img_dires:
        img=cv2.imread(img_dire)
        gray_img=bgr2gray(img)
        test_arr=[1024,512,256,128,64,32,16,4,2,1][::-1]
        for siz in test_arr:
            faces_rect = haar_cascade.detectMultiScale(gray_img, scaleFactor=1.1, minNeighbors=9)
            test_faces_rect=haar_cascade.detectMultiScale(gray_img, scaleFactor=1.1, minNeighbors=9,minSize=(siz,siz))
            if len(faces_rect)!=0 and len(test_faces_rect)==0:
                min_siz.append(siz)
    
    return min(min_siz)
        
        

### Function the maximum size of detected face size 

In [63]:
def maxFace(img_dires):
    max_siz=[]
    for img_dire in img_dires:
        img=cv2.imread(img_dire)
        gray_img=bgr2gray(img)
        test_arr=[1024,512,256,128,64,32,16,4,2,1]
        for siz in test_arr:
            faces_rect = haar_cascade.detectMultiScale(gray_img, scaleFactor=1.1, minNeighbors=9)
            test_faces_rect=haar_cascade.detectMultiScale(gray_img, scaleFactor=1.1, minNeighbors=9,maxSize=(siz,siz))
            if len(faces_rect)!=0 and len(test_faces_rect)==0:
                max_siz.append(siz)
    
    return max(max_siz)
        
        

# Testing the cascade classifier on custom made Dataset

In [12]:
cus_test_dires=glob(paths['custom_p_test']+'/*jpg')

In [13]:
g_t,_,_=test(classifier=pretrained_cascade,dire=cus_test_dires)

In [16]:
pred_count,e_t,img_len=test(classifier=haar_cascade,dire=cus_test_dires)

In [18]:
## Accuracy of the custom_face detection
print('Accuracy: {}%'.format(get_acc(g_t,pred_count)))

Accuracy: 32.0%


In [19]:
### Average time per face
print('Average time per face : {:03f} sec.'.format(time_per_face(pred_count,e_t)))

Average time per face : 0.131842 sec.


In [17]:
### Average time per Image
print('Average time per Image : {:03f} sec.'.format(time_per_face(img_len,e_t)))

Average time per Image : 0.125185 sec.


In [73]:
siz=minFace(cus_test_dires)
print('minimum size of face to be detected in custom datasets: ({},{})'.format(siz,siz))

minimum size of face to be detected in custom datasets: (64,64)


In [74]:
siz=maxFace(cus_test_dires)
print('maximum size of face to be detected in custom datasets: ({},{})'.format(siz,siz))

maximum size of face to be detected in custom datasets: (512,512)


In [22]:
## saving the result of all detected and failure images in Result/custom
for img_dire in cus_test_dires:
    img=cv2.imread(img_dire)
    img_name=img_dire.split('/')[-1].split('.')[0]
    gray_img=bgr2gray(img)
    faces_rect = haar_cascade.detectMultiScale(gray_img, scaleFactor=1.1, minNeighbors=9)
    for (x, y, w, h) in faces_rect:
        cv2.rectangle(img, (x, y), (x+w, y+h), (0, 255, 0), 2)
    save_dire=join(paths['res_cus'],'{}.jpg'.format(img_name))
    cv2.imwrite(save_dire,img)
    
    

## Testing on the public dataset

In [19]:
pub_test_dires=glob(paths['public_p_test']+'/*jpg')

In [21]:
g_t_p,_,_=test(classifier=pretrained_cascade,dire=pub_test_dires)

In [20]:
pred_count_pub,e_t_pub,img_len_pub=test(classifier=haar_cascade,dire=pub_test_dires)

In [24]:
## Accuracy of the Public_face detection
print('Accuracy on Public dataset: {}%'.format(get_acc(g_t_p,pred_count_pub)))

Accuracy on Public dataset: 73.41082455503087%


In [25]:
### Average time per face 
print('Average time per face on Public dataset: {:03f} sec.'.format(time_per_face(pred_count_pub,e_t_pub)))
## As each image is single image time for image per time and face per time is same

Average time per face on Public dataset: 0.002568 sec.


In [75]:
### minimum size of detectable face 
siz=minFace(pub_test_dires)
print('minimum face size detected in public(celeba) dataset ({},{}).'.format(siz,siz))

minimum face size detected in public(celeba) dataset (64,64).


In [76]:
### minimum size of detectable face 
siz=maxFace(pub_test_dires)
print('maximum face size detected in public(celeba) dataset ({},{}).'.format(siz,siz))

maximum face size detected in public(celeba) dataset (128,128).


In [33]:
## saving the result of all detected and failure images in Result/
for img_dire in pub_test_dires:
    img=cv2.imread(img_dire)
    img_name=img_dire.split('/')[-1].split('.')[0]
    gray_img=bgr2gray(img)
    faces_rect = haar_cascade.detectMultiScale(gray_img, scaleFactor=1.1, minNeighbors=9)
    for (x, y, w, h) in faces_rect:
        cv2.rectangle(img, (x, y), (x+w, y+h), (0, 255, 0), 2)
    save_dire=join(paths['res_pub'],'{}.jpg'.format(img_name))
    cv2.imwrite(save_dire,img)
    
    